In [ ]:
# Get reference to experiment
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()

my_experiment = Experiment(ws, 'auto-ml-deployment-experiment')

In [ ]:
# Get first run (in reverse chronological order)
run = next(my_experiment.get_runs())

print(f"Run {run.id} is {run.status}")

In [ ]:
from azureml.train.automl.run import AutoMLRun
automl_run = AutoMLRun(my_experiment, run.id)
# Zero is the Run and 1 is the best model
best_run, best_model = automl_run.get_output()

In [ ]:
best_run.download_files(prefix='outputs/', output_directory='./')

In [ ]:
training_input = automl_run.get_details()['inputDatasets'][0]
validation_input = automl_run.get_details()['inputDatasets'][1]
print(training_input)

In [ ]:
training_ds= training_input['dataset']
validation_ds = validation_input['dataset']

In [ ]:
from azureml.train.automl.runtime.automl_explain_utilities import automl_setup_model_explanations

x_train = training_ds.drop_columns('target')
y_train = training_ds.keep_columns('target')
x_test = validation_ds.drop_columns('target')
automl_explainer_setup_obj = automl_setup_model_explanations(
                                            best_model, 
                                            X=x_train,y=y_train,
                                            task='regression',
                                            features=['0', '1', '2',
                                            '3', '4', '5', '6', '7',
                                            '8', '9'],
                                            X_test=x_test
                                            )

In [ ]:
from azureml.interpret import MimicWrapper

# Initialize the Mimic Explainer
mimic_explainer = MimicWrapper(ws, automl_explainer_setup_obj.automl_estimator,
                         explainable_model=automl_explainer_setup_obj.surrogate_model, 
                         init_dataset=automl_explainer_setup_obj.X_transform, run=automl_run,
                         features=automl_explainer_setup_obj.engineered_feature_names, 
                         feature_maps=[automl_explainer_setup_obj.feature_map],
                         classes=automl_explainer_setup_obj.classes,
                         explainer_kwargs=automl_explainer_setup_obj.surrogate_model_params)

In [ ]:
from azureml.interpret.scoring.scoring_explainer import TreeScoringExplainer, save

# Initialize the ScoringExplainer
scoring_explainer = TreeScoringExplainer(mimic_explainer.explainer, feature_maps=[automl_explainer_setup_obj.feature_map])

# Pickle scoring explainer locally
save(scoring_explainer, directory='outputs', exist_ok=True)

In [ ]:
# Upload explainer to best run
best_run.upload_file('outputs/scoring_explainer.pkl','outputs/scoring_explainer.pkl')

In [ ]:
# Name of the model
best_model.steps[1][0]

In [ ]:
import azureml.core

best_run.register_model(model_name='automl_diabetes',
                               model_path="outputs",
                               tags={'type': 'regression', 'model': best_model.steps[1][0]},
                               model_framework=f"AutoML",
                               model_framework_version=azureml.core.VERSION, # Although model may be trained with the latest version
                               datasets=[("training data", training_ds),("validation data", validation_ds)])


In [ ]:
# Feature names
# https://docs.microsoft.com/en-us/python/api/azureml-automl-runtime/azureml.automl.runtime.featurization.data_transformer.datatransformer?view=azure-ml-py#attributes
list(best_model[0].get_column_names_and_types.keys())